Desafio #1

Alumno: Yandri Uchuari Gallardo

In [1]:
%pip install numpy scikit-learn

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [3]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [4]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [5]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [6]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [7]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [8]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [9]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [10]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [11]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [12]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [13]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [14]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [15]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 9019, 9016, 8748])

In [16]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [17]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [18]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [19]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [20]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [21]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

In [34]:
import numpy as np
import random

target_names = newsgroups_train.target_names

#Semilla a usar
np.random.seed(19)

#Seleccionar 5 documentos al azar
random_docs_idx = np.random.choice(X_train.shape[0], size=5, replace=False)

count = 1
#Mostrar los 5 textos completos
for idx in random_docs_idx:
    print("Texto: ",count)
    print("="*80)
    print(f"Documento índice {idx} - Etiqueta: {target_names[y_train[idx]]}")
    print(newsgroups_train.data[idx])
    print("\n")
    count += 1


Texto:  1
Documento índice 5486 - Etiqueta: talk.politics.misc

So which are you advocating?

That You know Nothing About American History, 
Or that You Know Nothing About the Bible?

Is this a Restoration of the "Know Nothing" Party?

ciao
drieux

ps: what WAS the "Free Negro Sailor Act" about,
and what was the Supreme Court's Ruling On it... and
More Importantly, how does this Complicate the Mythology
that all blacks were slaves????




Texto:  2
Documento índice 3701 - Etiqueta: comp.windows.x
[again, poseted for a friend]

Okay, I got such great response from my last question (thanks, everyone), 
lets try again:

How can I tell if a colorcell is read-only?

I want to use any read-only cells that are already in my colormap
once I start running out of allocatable cells, using some arbitrary
nearest color matching scheme to get those cells that come closest to
a requested color.

Once I determine if a cell is read-only, I assume I can then just request
an XAllocColor() with the exact 

In [36]:
#Calcular similitud coseno con todos los documentos
similar_docs = {}
for idx in random_docs_idx:
    sim_scores = cosine_similarity(X_train[idx], X_train).flatten()
    # Ordenar por similitud (ignorando el mismo documento)
    similar_indices = sim_scores.argsort()[::-1][1:6]  # top 5
    similar_docs[idx] = {
        "texto": newsgroups_train.data[idx][:300].replace("\n", " ") + "...",
        "etiqueta": target_names[y_train[idx]],
        "similares": [
            {
                "indice": i,
                "similitud": sim_scores[i],
                "etiqueta": target_names[y_train[i]],
                "texto": newsgroups_train.data[i][:200].replace("\n", " ") + "..."
            }
            for i in similar_indices
        ]
    }

In [37]:

# 3. Mostrar resultados
for doc_idx, info in similar_docs.items():
    print("="*80)
    print(f"Documento base (índice {doc_idx}) - Etiqueta: {info['etiqueta']}")
    print(info['texto'])
    print("\nDocumentos más similares:")
    for sim in info['similares']:
        print(f"  -> (índice {sim['indice']}) similitud={sim['similitud']:.4f} "
              f"Etiqueta: {sim['etiqueta']}")
        print(f"     {sim['texto']}\n")

Documento base (índice 5486) - Etiqueta: talk.politics.misc
 So which are you advocating?  That You know Nothing About American History,  Or that You Know Nothing About the Bible?  Is this a Restoration of the "Know Nothing" Party?  ciao drieux  ps: what WAS the "Free Negro Sailor Act" about, and what was the Supreme Court's Ruling On it... and More Importan...

Documentos más similares:
  -> (índice 9623) similitud=0.2312 Etiqueta: talk.politics.mideast
     Accounts of Anti-Armenian Human Right Violations in Azerbaijan #012                  Prelude to Current Events in Nagorno-Karabakh          +---------------------------------------------------------+ ...

  -> (índice 3902) similitud=0.2083 Etiqueta: talk.religion.misc
        Are you your own master?  Do you have any habits that you cannot break? For one, you seem unable to master your lack of desire to understand even the slightest concept of the Bible.  Seems that ign...

  -> (índice 7464) similitud=0.2052 Etiqueta: soc.religi

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámetros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

In [38]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, classification_report

# =====================
# 1. Definir pipeline base
# =====================
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

# =====================
# 2. Definir espacio de parámetros para GridSearch
# =====================
parameters = [
    {
        'tfidf__ngram_range': [(1, 1), (1, 2)],    # Unigramas vs Unigramas+Bigramas
        'tfidf__stop_words': [None, 'english'],    # Con o sin stopwords
        'clf': [MultinomialNB()],
        'clf__alpha': [0.1, 0.5, 1.0]
    },
    {
        'tfidf__ngram_range': [(1, 1), (1, 2)],
        'tfidf__stop_words': [None, 'english'],
        'clf': [ComplementNB()],
        'clf__alpha': [0.1, 0.5, 1.0]
    }
]

# =====================
# 3. GridSearch con validación cruzada
# =====================
grid = GridSearchCV(pipeline, parameters, cv=3, n_jobs=-1, scoring='f1_macro')
grid.fit(newsgroups_train.data, newsgroups_train.target)

print("Mejores parámetros encontrados:")
print(grid.best_params_)
print(f"Mejor F1-score (cross-validation): {grid.best_score_:.4f}")

# =====================
# 4. Evaluar en conjunto de test
# =====================
y_pred = grid.predict(newsgroups_test.data)
print("Reporte clasificación en test:")
print(classification_report(newsgroups_test.target, y_pred, target_names=newsgroups_test.target_names))

f1_macro_test = f1_score(newsgroups_test.target, y_pred, average='macro')
print(f"F1-score macro en test: {f1_macro_test:.4f}")


Mejores parámetros encontrados:
{'clf': ComplementNB(), 'clf__alpha': 0.1, 'tfidf__ngram_range': (1, 2), 'tfidf__stop_words': 'english'}
Mejor F1-score (cross-validation): 0.7667
Reporte clasificación en test:
                          precision    recall  f1-score   support

             alt.atheism       0.34      0.48      0.39       319
           comp.graphics       0.73      0.72      0.73       389
 comp.os.ms-windows.misc       0.69      0.62      0.65       394
comp.sys.ibm.pc.hardware       0.66      0.69      0.67       392
   comp.sys.mac.hardware       0.79      0.71      0.75       385
          comp.windows.x       0.80      0.80      0.80       395
            misc.forsale       0.78      0.79      0.78       390
               rec.autos       0.81      0.74      0.77       396
         rec.motorcycles       0.82      0.77      0.79       398
      rec.sport.baseball       0.93      0.84      0.88       397
        rec.sport.hockey       0.88      0.94      0.91       3

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 1. Crear y ajustar el vectorizador (si no está ya creado)
tfidf_vect = TfidfVectorizer()
X_train = tfidf_vect.fit_transform(newsgroups_train.data)

# 2. Transponer matriz documento-término (término-documento)
X_terms = X_train.T

# 3. Diccionario inverso índice → palabra
idx2word = {v: k for k, v in tfidf_vect.vocabulary_.items()}

# 4. Elegir palabras manualmente
selected_words = ["space", "god", "windows", "hockey", "gun"]

# 5. Calcular similaridad de cada palabra con el resto
for word in selected_words:
    if word not in tfidf_vect.vocabulary_:
        print(f"'{word}' no está en el vocabulario")
        continue

    word_idx = tfidf_vect.vocabulary_[word]
    word_vector = X_terms[word_idx]

    sim_scores = cosine_similarity(word_vector, X_terms).flatten()
    # Ordenar excluyendo la propia palabra
    similar_indices = sim_scores.argsort()[::-1][1:6]

    print("="*80)
    print(f"Palabra base: '{word}'")
    for idx in similar_indices:
        similar_word = idx2word[idx]
        print(f"  -> {similar_word:20s} similitud={sim_scores[idx]:.4f}")


Palabra base: 'space'
  -> nasa                 similitud=0.3304
  -> seds                 similitud=0.2966
  -> shuttle              similitud=0.2928
  -> enfant               similitud=0.2803
  -> seti                 similitud=0.2465
Palabra base: 'god'
  -> jesus                similitud=0.2688
  -> bible                similitud=0.2616
  -> that                 similitud=0.2560
  -> existence            similitud=0.2548
  -> christ               similitud=0.2511
Palabra base: 'windows'
  -> dos                  similitud=0.3037
  -> ms                   similitud=0.2320
  -> microsoft            similitud=0.2219
  -> nt                   similitud=0.2140
  -> for                  similitud=0.1930
Palabra base: 'hockey'
  -> ncaa                 similitud=0.2743
  -> nhl                  similitud=0.2653
  -> affiliates           similitud=0.2480
  -> xenophobes           similitud=0.2426
  -> sportschannel        similitud=0.2228
Palabra base: 'gun'
  -> guns                 simil

**Conclusion**
En este desafío aprendi a transformar texto en vectores mediante bolsa de palabras y TF–IDF, y a entrenar modelos probabilísticos como Naïve Bayes para clasificación de documentos. 

Pude comprobar que el modelo puede identificar correctamente las categorías del dataset y que TF–IDF ofrece representaciones más ricas que mejoran el rendimiento respecto al conteo simple.

**Recomendación**

Recomiendo seguir testeando otros algoritmos de clasificación (como SVM o redes neuronales ligeras) y ampliar la experimentación con preprocesamiento de texto (eliminación de stopwords, lematización). 

Además, resulta útil aplicar estos métodos a un corpus en español para comprobar la adaptabilidad del enfoque a distintos idiomas.